In [1]:
import scanpy as sc
import decoupler as dc
import pandas as pd
import os
import matplotlib.pyplot as plt
# Plotting options, change to your liking
sc.settings.set_figure_params(dpi=200, frameon=False)
sc.set_figure_params(dpi=200)
sc.set_figure_params(figsize=(4, 4))

sc.settings.verbosity = 3
# sc.logging.print_versions()

import warnings
warnings.filterwarnings("ignore")

from tqdm import tqdm
#####---------------------------------------------------------------------------------------------------------#####
##### CONFIGURATIONS
outdir = "/media/hieunguyen/CRC1382H/CRC1382/outdir"
PROJECT = "20231018_SAlBounny.filter_contaminated_cells.clusterRes_0.5"
path_to_main_output = os.path.join(outdir, "20231018_SAlBounny", "data_analysis")
path_to_01_output = os.path.join(path_to_main_output, "01_output")
path_to_02_output = os.path.join(path_to_main_output, "02_output")
path_to_03_output = os.path.join(path_to_main_output, "03_output")
path_to_08_output = os.path.join(path_to_main_output, "08_output")
path_to_10_output = os.path.join(path_to_main_output, "10_output")
os.system("mkdir -p {}".format(path_to_10_output))
path_to_seurat2anndata = os.path.join(path_to_08_output, "seurat2anndata")

os.system("mkdir -p {}".format(os.path.join(path_to_10_output, "violinplot") ))
os.system("mkdir -p {}".format(os.path.join(path_to_10_output, "umap_acts") ))
adata = sc.read_h5ad(os.path.join(path_to_seurat2anndata, '{}.h5ad'.format(PROJECT)))

adata.obs["seurat_clusters"] = ["cluster{}".format(item) for item in adata.obs["seurat_clusters"].values]

In [ ]:
sc.pl.umap(adata, color = "seurat_clusters")

In [3]:
net = dc.get_collectri(organism='mouse', split_complexes=False)

In [ ]:
dc.run_ulm(
    mat = adata,
    net = net,
    source = 'source',
    target = 'target',
    weight = 'weight',
    verbose = True,
    use_raw = False)

In [ ]:
adata.obsm['collectri_ulm_estimate'] = adata.obsm['ulm_estimate'].copy()
adata.obsm['collectri_ulm_pvals'] = adata.obsm['ulm_pvals'].copy()
adata

In [6]:
acts = dc.get_acts(adata, obsm_key='ulm_estimate')


In [ ]:
for input_gene in tqdm(acts.var.index):
    with plt.rc_context():  # Use this to set figure params like size and dpi
        sc.pl.umap(acts, color=[input_gene, 'seurat_clusters'], cmap='RdBu_r', vcenter=0, show = False, size = 20)
        plt.savefig(os.path.join(path_to_10_output, "umap_acts", "UMAP_{}.png".format(input_gene)), bbox_inches="tight")
        sc.pl.violin(acts, keys=[input_gene], groupby='seurat_clusters', rotation=90, show  = False)
        plt.savefig(os.path.join(path_to_10_output, "violinplot", "violin_plot_{}.png".format(input_gene)), bbox_inches="tight")

In [ ]:
df = dc.rank_sources_groups(acts, groupby='seurat_clusters', reference='rest', method='t-test_overestim_var')

In [ ]:
n_markers = 10
source_markers = df.groupby('group').head(n_markers).groupby('group')['names'].apply(lambda x: list(x)).to_dict()
#
source_markers_df = pd.DataFrame.from_dict(source_markers, orient = "index")

source_markers_df

In [ ]:
# sc.tl.dendrogram(adata, groupby="seurat_clusters")
sc.pl.matrixplot(acts, source_markers, 'seurat_clusters', 
                 dendrogram=False, 
                 standard_scale='var', 
                 colorbar_title='Z-scaled scores', 
                 cmap='RdBu_r')

In [ ]:
# dc.plot_network(
#     net=net,
#     n_sources=['Pax5', 'Ebf1', 'Rfxap'],
#     n_targets=15,
#     node_size=100,
#     s_cmap='white',
#     t_cmap='white',
#     c_pos_w='darkgreen',
#     c_neg_w='darkred',
#     figsize=(5, 5)
# )